In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [ ]:
import parkit as pk

pk.import_site('C:\\Users\\rdpuser\\Desktop\\test')

In [ ]:
# Smoke Tests

In [ ]:
collection_tests = pk.bind_task('collection_tests')

In [ ]:
collection_tests(1000, sync = True)

In [ ]:
pk.enable_tasks()

In [ ]:
collection_tests(1000)

In [ ]:
executions = []
for i in range(16):
    executions.append(collection_tests(1000))

In [ ]:
pk.wait(lambda: all(e.done for e in executions))
assert all(e.result for e in executions)

In [ ]:
pk.disable_tasks()

In [ ]:
# File Tests

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
rows = []
for i in range(1000000):
    rows.append(dict(a = i, b = i))
df = pd.DataFrame(rows)

In [ ]:
f = pk.File(mode = 'wb')

In [ ]:
with f:
    df.to_feather(f)

In [ ]:
f.mode = 'rb'
with f:
    assert df.equals(pd.read_feather(f))

In [ ]:
a = np.zeros((100,100))

In [ ]:
f = pk.File(mode = 'wb+')
with f:
    np.save(f, a)

In [ ]:
f.mode = 'rb+'
with f:
    assert np.array_equal(a, np.load(f))

In [ ]:
pk.gc()